In [1]:
#Tạo đường dẫn chung để đọc utils
import sys
import os

#Tạo đường dẫn để gọi đến utils dễ hơn
project_root = os.path.abspath(os.path.join("..", '..', 'src'))
sys.path.append(project_root)

print(f"Đã thêm đường dẫn: {project_root}")
print(f"Thư mục hiện tại: {os.getcwd()}")

# Utils
from utils import load_chunks_from_json
from utils import save_chunks_to_json
from utils import bm25_tokenize, text_to_sparse_vector_bm25

Đã thêm đường dẫn: c:\DACNTT2526\IT_Project_2526\src
Thư mục hiện tại: c:\DACNTT2526\IT_Project_2526\data\TuTuongHoChiMinh


In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTTextLine, LTChar
from pypdf import PdfReader

from sentence_transformers import SentenceTransformer


from rank_bm25 import BM25Okapi

import unicodedata
import torch
import numpy as np
import re

# ***Chunk văn bản***

In [ ]:
def get_text_size_TrietHoc(pdf_path, total_pages):
    '''
        Hàm được sử dụng để kiểm tra xem toàn bộ các size text có trong tài liệu, chuẩn bị cho việc chunking
        Các tham số:
            - pdf_path: đường dẫn đến file pdf của môn Lịch sử đảng
        
        Hàm sẽ trả về 1 set chứa toàn bộ các text_size của tài liệu
    '''
    #Duyệt qua từng trang trong file pdf, bắt đầu từ trang 2 (bỏ qua trang bìa)
    size_set = set()
    for page_layout in extract_pages(pdf_path, page_numbers=range(2, total_pages)):
        
        #Duyệt qua từng phần tử trong trang
        for element in page_layout:
            if isinstance(element, LTTextContainer):

                # Duyet qua từng dòng chữ trong phần tử
                for text_line in element:
                    if isinstance(text_line, LTTextLine):
                        
                        #Lấy ra từng kí tự bên trong dòng chữ
                        for obj in text_line:
                            if isinstance(obj, LTChar):

                                #Lấy ra thông tin của kí tự
                                font = obj.fontname
                                text = obj.get_text()
                                text_size = obj.size

                                size_set.add(text_size)
    
    return size_set

# Xây dựng các hàm cần thiết cho quá trình xử lý văn bản

In [3]:
def parse_line(line):
    """
    Tách định danh và tiêu đề từ các dòng đề mục.

    Ví dụ:
    - 'Phần II  Những nguyên lý...' → ('Phần II', 'Những nguyên lý...')
    - 'Chương V  Vật chất...'       → ('Chương V', 'Vật chất...')
    - 'II- Nguồn gốc...'            → ('II-', 'Nguồn gốc...')
    - '3. Kết cấu...'               → ('3.', 'Kết cấu...')
    - 'b) Theo chiều sâu...'        → ('b)', 'Theo chiều sâu...')

    Nếu không khớp, trả về ('None', original_line)
    """

    line = line.strip()

    pattern = re.compile(
        r"""^
        (
            (Phần|Chương)\s+[IVXLCDM\d]+            # Phần II, Chương V
            |
            [IVXLCDM]+[-\.]                         # II- hoặc II.
            |
            \d+[\.\)]                               # 3. hoặc 3)
            |
            [a-zA-Z][\.\)]                          # a) hoặc A.
        )
        \s+(.+)                                     # phần còn lại là tiêu đề
        """, re.VERBOSE | re.IGNORECASE
    )

    match = pattern.match(line)
    if match:
        number = match.group(1).strip().rstrip(".-)")  # loại . - )
        title = match.group(3).strip()
        return number, title
    else:
        return "", line

In [4]:
def normalize_text(text):
    """
    Chuyển chuỗi thành lowercase, xóa dấu, và xóa toàn bộ khoảng trắng.
    """
    # Lowercase
    text = text.lower()

    # Bỏ dấu
    text = unicodedata.normalize('NFD', text)
    text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')

    # Xóa toàn bộ khoảng trắng (space, tab, newline)
    text = re.sub(r'\s+', '', text)

    return text

In [5]:
#Hàm dùng để nhận thông tin từ raw chunks, chuyển đổi sang DENSE vector để chuẩn bị nạp vào Database
def TT_HCM_raw_to_dense(chapter, section, subsection, sub_subsection, content, model, max_chars = 2048, global_chunk_counter=None):

    #Tách các đề mục sang số mục và tiêu đề
    chapter_number, chapter_title = parse_line(chapter)
    section_number, section_title = parse_line(section)
    subsection_number, subsection_title = parse_line(subsection)
    sub_subsection_number, sub_subsection_title = parse_line(sub_subsection)

    chunk_type = 'EXERCISES' if ("Câu hỏi ôn tập" in section or "câu hỏi ôn tập" in section) else 'THEORY'

    from langchain.text_splitter import RecursiveCharacterTextSplitter

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_chars,
        chunk_overlap=300,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    content_chunks = splitter.split_text(content)

    result = []
    for i, content_piece in enumerate(content_chunks):
        if global_chunk_counter is not None:
            # Sử dụng global counter để đảm bảo hoàn toàn unique
            unique_id = f"TT_HCM_chunk_{global_chunk_counter['count']:05d}"
            global_chunk_counter['count'] += 1
        else:
            # Fallback về cách cũ nếu không có global counter
            unique_id = f"TT_HCM_{normalize_text(chapter_number) if chapter_number else 0}_{section_number if section_number else 0}_{subsection_number if section_number else 0}_{sub_subsection_number if sub_subsection_number else 0}_{i}"  
        chunk = {
            "id": f"{unique_id}",
            "values": model.encode(content).tolist(),
            "metadata": {
                "subject": "Tư tưởng Hồ Chí Minh",
                "chapter": chapter_number,
                "chapter_title": chapter_title,
                "section": section_number,
                "section_title": section_title,
                "subsection": subsection_number,
                "subsection_title": subsection_title,
                "sub_subsection": sub_subsection_number,
                "sub_subsection_title": sub_subsection_title,
                "content": content_piece,
                "tokens": len(content_piece),
                "type": chunk_type
            }
        }
        result.append(chunk)

    return result

In [6]:
def bm25_tokenize(text):
    return text.lower().split()

def text_to_sparse_vector_bm25(text, bm25, vocabulary):
    tokens = bm25_tokenize(text)
    vector = np.zeros(len(vocabulary))
    for i, word in enumerate(vocabulary):
        idf = bm25.idf.get(word, 0)
        tf = tokens.count(word)
        vector[i] = idf * tf
    indices = vector.nonzero()[0].tolist()
    values = vector[indices].tolist()
    return {"indices": indices, "values": values}

In [6]:
#Hàm dùng để nhận thông tin từ raw chunks, chuyển đổi sang SPARSE vector để chuẩn bị nạp vào Database
def TT_HCM_raw_to_sparse(chapter, section, subsection, sub_subsection, content, bm25 = None, vocabluary = None, max_chars = 2048, global_chunk_counter=None):

    #Tách các đề mục sang số mục và tiêu đề
    chapter_number, chapter_title = parse_line(chapter)
    section_number, section_title = parse_line(section)
    subsection_number, subsection_title = parse_line(subsection)
    sub_subsection_number, sub_subsection_title = parse_line(sub_subsection)

    chunk_type = 'EXERCISES' if ("Câu hỏi ôn tập" in section or "câu hỏi ôn tập" in section) else 'THEORY'

    from langchain.text_splitter import RecursiveCharacterTextSplitter

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_chars,
        chunk_overlap=300,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    content_chunks = splitter.split_text(content)

    result = []
    for i, content_piece in enumerate(content_chunks):
        if global_chunk_counter is not None:
            # Sử dụng global counter để đảm bảo hoàn toàn unique
            unique_id = f"TT_HCM_chunk_{global_chunk_counter['count']:05d}"
            global_chunk_counter['count'] += 1
        else:
          # Fallback về cách cũ nếu không có global counter
            unique_id = f"TT_HCM_{normalize_text(chapter_number) if chapter_number else 0}_{section_number if section_number else 0}_{subsection_number if section_number else 0}_{sub_subsection_number if sub_subsection_number else 0}_{i}"
        chunk = {
            "id": f"{unique_id}",
            "sparse_values": text_to_sparse_vector_bm25(content_piece.strip(), bm25, vocabluary),
            "metadata": {
                "subject": "Tư tưởng Hồ Chí Minh",
                "chapter": chapter_number,
                "chapter_title": chapter_title,
                "section": section_number,
                "section_title": section_title,
                "subsection": subsection_number,
                "subsection_title": subsection_title,
                "sub_subsection": sub_subsection_number,
                "sub_subsection_title": sub_subsection_title,
                "content": content_piece,
                "tokens": len(content_piece),
                "type": chunk_type
            }
        }
        result.append(chunk)

    return result

# Xử lý tách văn bản thành các chunks

In [ ]:
#Phân chia các tiêu đề
# SUBSECTION_PATTERN = re.compile(r"^\s*\d+[\.\)]")    # 1. hoặc 2)
# SUBSECTION_PATTERN_CHAP2 = re.compile(r"^\s*[IVX]+-", re.IGNORECASE)   # 1. hoặc 2)
# SUB_SUBSECTION_PATTERN = re.compile(r"^\s*[a-zA-Z][\.\)]")  # a), b.

In [ ]:
#Hàm đung để kiểm tra xem tiêu đề hiện tại có phải sub section không
# def is_subsection(line_chars, chapter_title):
#     line = ''.join([obj.get_text() for obj in line_chars]).strip()
#     if "Chương II  " in chapter_title:
#         return bool(SUBSECTION_PATTERN_CHAP2.match(line))
#     return bool(SUBSECTION_PATTERN.match(line))

# #Hàm đung để kiểm tra xem tiêu đề hiện tại có phải subsub section không
# def is_sub_subsection(line_chars, chapter_title):
#     line = ''.join([obj.get_text() for obj in line_chars]).strip()
#     if "Chương II " in chapter_title:
#         return bool(SUBSECTION_PATTERN.match(line))
#     return bool(SUB_SUBSECTION_PATTERN.match(line))

# #Hàm đung để kiểm tra xem hiện tại có phải câu hỏi ôn tập hay không
# def is_questions(line_chars):
#     line = ''.join([obj.get_text() for obj in line_chars]).strip()
#     return ("Câu hỏi ôn tập" in line or "câu hỏi ôn tập" in line)

In [7]:
# Hàm trả về các raw chunks từ file PDF, đã được tối ưu cho tài liệu Tư tưởng Hồ Chí Minh
def extract_raw_chunks_ttHCM(pdf_path, total_pages):
    """
    Trích xuất các đoạn nội dung từ file PDF, đã được tối ưu để:
    1. Gộp tiêu đề chương đa dòng.
    2. Loại bỏ phần "TÀI LIỆU THAM KHẢO".
    3. Loại bỏ các chú thích ở cuối trang.
    4. Loại bỏ các chỉ mục footnote (ví dụ: ...word1.) trong nội dung.
    
    Trả về danh sách các dict: 
    {"chapter": ..., "section": ..., "sub_section": ..., "sub_subsection": ..., "content": ...}
    """

    base_patterns = [
        r'Downloaded by', r'Downloaded from', r'https?://', r'www\.',
        r'\S+@\S+\.\S+', r'^\s*Page\s*\d+\s*$', r'^[\W_]{3,}$',
        r'^[A-Za-z0-9]{6,}\|\d{3,}$', r'^\s*Downloaded\b', r'DOWNLOADED\b',
        r'lOMoARcPSD'
    ]
    compiled_patterns = [re.compile(p, re.I) for p in base_patterns]

    def is_garbage_line(s):
        if not s or not s.strip():
            return True
        s = s.replace('\x0c','').strip()
        if len(s) <= 2 and re.match(r'^[\d\W_]+$', s):
            return True
        for cp in compiled_patterns:
            if cp.search(s):
                return True
        non_alnum = sum(1 for ch in s if not ch.isalnum() and not ch.isspace())
        if len(s) > 0 and (non_alnum / len(s)) > 0.6:
            return True
        return False
    
    footnote_re = re.compile(r'^\s*\d+\s+.*\b(Nxb|NXB|Hà Nội|Hanoi|tr\.|t\.)\b|,\s*\d{4}\b', re.I)
    chapter_pattern = re.compile(r"^Chương ([IVXLCDM]+|\d+)\s*$", re.IGNORECASE)
    section_pattern = re.compile(r"^[IVXLCDM]+\.\s*")
    sub_section_pattern = re.compile(r"^\d+\.\s*")
    sub_sub_section_pattern = re.compile(r"^[a-z]\.\s*")

    raw_chunks = []
    is_content = False
    in_references_section = False  # Flag trạng thái cho mục TÀI LIỆU THAM KHẢO

    curr_chapter_line = ""
    curr_section_line = ""
    curr_sub_section_line = ""
    curr_sub_sub_section_line = ""
    curr_content_line = ""

    page_range = range(5, total_pages)

    for page_layout in extract_pages(pdf_path, page_numbers=page_range):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                for text_line in element:
                    if not isinstance(text_line, LTTextLine):
                        continue
                        
                    line_text = text_line.get_text().replace('\x0c','').strip()
                    if is_garbage_line(line_text) or line_text.isdigit():
                        continue

                    first_char = next(iter(text_line), None)
                    font_name = ""
                    font_size = first_char.size
                    if isinstance(first_char, LTChar):
                        font_name = first_char.fontname

                    is_bold = "Bold" in font_name
                    is_italic = "Italic" in font_name

                    # Loại bỏ chú thích cuối trang dựa trên font size 
                    is_small_font = font_size < 12
                    if is_small_font or footnote_re.search(line_text):
                        continue

                    # Xử lý trạng thái "TÀI LIỆU THAM KHẢO" 
                    is_new_header = any([p.match(line_text) for p in [chapter_pattern, section_pattern]])
                    if in_references_section:
                        if is_new_header:
                            in_references_section = False # Thoát khỏi chế độ bỏ qua
                        else:
                            continue # Bỏ qua các dòng trong mục tham khảo

                    if line_text.strip().upper() == 'TÀI LIỆU THAM KHẢO':
                        save_current_chunk()
                        in_references_section = True
                        continue

                    def save_current_chunk():
                        nonlocal is_content, curr_content_line
                        if is_content and curr_content_line.strip():
                            raw_chunks.append({
                                "chapter": curr_chapter_line.strip(),
                                "section": curr_section_line.strip(),
                                "sub_section": curr_sub_section_line.strip(),
                                "sub_subsection": curr_sub_sub_section_line.strip(),
                                "content": curr_content_line.strip()
                            })
                        is_content = False
                        curr_content_line = ""

                    # 1. Kiểm tra Chương
                    if chapter_pattern.match(line_text) and is_bold:
                        save_current_chunk()
                        curr_chapter_line = line_text
                        curr_section_line = ""
                        curr_sub_section_line = ""
                        curr_sub_sub_section_line = ""
                    
                    # 2. Kiểm tra Mục (I, II, ...)
                    elif section_pattern.match(line_text) and is_bold:
                        save_current_chunk()
                        curr_section_line = line_text
                        curr_sub_section_line = ""
                        curr_sub_sub_section_line = ""

                    # 3. Kiểm tra Mục con (1, 2, ...)
                    elif sub_section_pattern.match(line_text) and is_bold:
                        save_current_chunk()
                        curr_sub_section_line = line_text
                        curr_sub_sub_section_line = ""

                    # 4. Kiểm tra Mục con cấp 2 (a, b, ...)
                    elif sub_sub_section_pattern.match(line_text) and is_bold and is_italic:
                        save_current_chunk()
                        curr_sub_sub_section_line = line_text
                        
                    # 5. Xử lý nội dung hoặc phần còn lại của tiêu đề chương
                    else:
                        # Nếu đã có chương, chưa có mục, và dòng hiện tại là IN HOA + in đậm
                        # -> coi nó là một phần của tiêu đề chương.
                        # Thêm điều kiện len(line_text.split()) > 2 để chỉ gộp những cụm từ dài
                        # Tương tự với mục và mục con.
                        if (curr_chapter_line and not curr_section_line and
                                is_bold and line_text.isupper() and len(line_text.split()) > 2):
                            curr_chapter_line += " " + line_text
                        elif (curr_section_line and not curr_sub_section_line and
                                is_bold):
                            curr_section_line += " " + line_text
                        elif (curr_sub_section_line and not curr_sub_sub_section_line and
                                is_bold):
                            curr_sub_section_line += " " + line_text
                        elif (curr_sub_sub_section_line and
                                is_bold and is_italic):
                            curr_sub_sub_section_line += " " + line_text
                        else:
                            # Ngược lại, coi là nội dung bình thường
                            if footnote_re.search(line_text):
                                continue
                            # Regex để loại bỏ các số footnote, ví dụ: "word”1." -> "word”."
                            # Quy tắc: tìm một ký tự (chữ hoặc dấu ngoặc kép), theo sau là một chữ số (1-9), 
                            # và kết thúc bằng một ký tự khoảng trắng hoặc dấu câu.
                            # Thay thế bằng ký tự đầu và cuối, bỏ qua chữ số ở giữa.
                            line_text = re.sub(r'([a-zA-Zà-ỹÀ-Ỹ”\'"])([1-9])([\s\.,\);]|$)', r'\1\3', line_text)
                            is_content = True
                            curr_content_line += line_text + " "

    if curr_content_line.strip():
        raw_chunks.append({
            "chapter": curr_chapter_line.strip(),
            "section": curr_section_line.strip(),
            "sub_section": curr_sub_section_line.strip(),
            "sub_subsection": curr_sub_sub_section_line.strip(),
            "content": curr_content_line.strip()
        })

    return raw_chunks

# ***Tạo ra Dense Vector và Sparse Vector từ Raw***

## Tạo Dense Vector

In [8]:
def chunk_with_embedding(raw_chunks, embedding_model):
    """
    Nhận vào danh sách raw_chunks, sau đó gọi hàm split_chunk_langchain để xử lý nhỏ ra và thêm embedding nếu cần.
    """
    final_chunks = []
    global_chunk_counter = {'count': 0}  # Khởi tạo global counter
    for chunk in raw_chunks:
        split_chunks = TT_HCM_raw_to_dense(
            chunk['chapter'],
            chunk['section'],
            chunk['sub_section'],
            chunk['sub_subsection'],
            chunk['content'],
            max_chars=2048,
            model=embedding_model,
            global_chunk_counter=global_chunk_counter
        )
        final_chunks.extend(split_chunks)

    return final_chunks

## Tạo Sparse Vector

In [9]:
def chunk_with_sparse(raw_chunks, bm25, vocabulary):
    """
    Nhận vào danh sách raw_chunks, sau đó gọi hàm split_chunk_langchain để xử lý nhỏ ra và thêm embedding nếu cần.
    """
    final_chunks = []
    global_chunk_counter = {'count': 0}  # Khởi tạo global counter
    for chunk in raw_chunks:
        split_chunks = TT_HCM_raw_to_sparse(
            chunk['chapter'],
            chunk['section'],
            chunk['sub_section'],
            chunk['sub_subsection'],
            chunk['content'],
            bm25=bm25,
            vocabluary=vocabulary,
            global_chunk_counter=global_chunk_counter
        )
        final_chunks.extend(split_chunks)

    return final_chunks

# ***Tạo các file json cần thiết***

In [9]:
import json
import os

def save_chunks_to_json(chunks, output_path):
    """
    Lưu danh sách các chunk (list of dict) ra file JSON.
    """
    os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Tạo folder nếu chưa có

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2)

    print(f"✅ Đã lưu {len(chunks)} chunks vào: {output_path}")

In [7]:
# Đọc data từ file json
def load_chunks_from_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        chunks = json.load(f)
    return chunks

# ***Main***

In [11]:
pdf_path = "./Tu_tuong_Ho_Chi_Minh.pdf"
# API_KEY = os.getenv("PINECONE_API_KEY")
# HOST_DENSE = os.getenv("HOST_DENSE")
# HOST_SPARSE = os.getenv("HOST_SPARSE")

# Lấy số trang trong file PDF
reader = PdfReader(pdf_path)
total_pages = len(reader.pages)
print("Số trang trong PDF:", total_pages)

# Lấy ra toàn bộ fontsize của file PDF
# size_set = get_text_size_TrietHoc(pdf_path, total_pages)
# print(f'size set: {size_set}')

# - Phân loại các nội dung (Format của môn Triết học sẽ theo thứ tự: Phần -> Chương -> I, II, III,... -> 1, 2,... -> a, b, c,....):
#     - Kích thước chữ >= 22: Các phần và tiêu đề của các phần đó
#     - Kích thước chữ == 18: Chương và các tiêu đề của Chương
#     - Kích thước chữ == 14 (> 13.5 and < 14): Các mục I, II, III (Có thể là ABC đối với chương II và 123 đói với chương IV)
#     - Kích thước chữ == 13 (13 <= text_size < 13.1): Các mục con như 1, 2, 3,... (Đối với chương 2 sẽ là I, II, III,...) và các mục nhỏ hơn như a, b, c....
#     - Kích thước chữ == 12 (11.6 <= text_size < 12,4): Nội dung của môn

# Đọc file PDF, chuyển qua RAW chunks
# TT_HCM_raw_chunks = extract_raw_chunks_ttHCM(pdf_path, total_pages)
# save_chunks_to_json(TT_HCM_raw_chunks, r"./chunks/Tu_tuong_HCM_raw.json")

TT_HCM_raw_chunks = load_chunks_from_json(r"./chunks/Tu_tuong_HCM_raw.json")
# Đưa RAW chunks vào chuyển đổi sang Dense Vector
embedding_model = SentenceTransformer("AITeamVN/Vietnamese_Embedding")
embedding_model.max_seq_length = 2048
TT_HCM_dense_vector = chunk_with_embedding(TT_HCM_raw_chunks, embedding_model)

# Đưa RAW chunks vào chuyển đổi sang Sparse Vector
# corpus_texts = [chunk["content"] for chunk in TT_HCM_raw_chunks]
# tokenized_corpus = [bm25_tokenize(text) for text in corpus_texts]
# bm25 = BM25Okapi(tokenized_corpus)
# vocabulary = list(bm25.idf.keys())
# TT_HCM_sparse_vector = chunk_with_sparse(TT_HCM_raw_chunks, bm25, vocabulary)


#Lưu các Vector vừa tạo được vào file Json

#Lưu Dense Vector
save_chunks_to_json(TT_HCM_dense_vector, r"./chunks/TT_HCM_Dense.json")

#Lưu Sparse Vector
# save_chunks_to_json(TrietHoc_sparse_vector, r"./TrietHoc_Sparse.json")                    

# Đọc file Json 
# tu_tuong_HCM_dense_vector = load_chunks_from_json(r"./TT_HCM_Dense.json")
# tu_tuong_HCM_sparse_vector = load_chunks_from_json(r"./TT_HCM_Sparse.json")


# Upsert Vector lên các Database
# from pinecone.grpc import PineconeGRPC as Pinecone
# pc = Pinecone(api_key=API_KEY)

# dense_index = pc.Index(host=HOST_DENSE)
# dense_index.upsert(
#   vectors = tu_tuong_HCM_dense_vector,
#   namespace="tu-tuong-HCM"
# )

# sparse_index = pc.Index(host=HOST_SPARSE)
# sparse_index.upsert(
#   vectors = tu_tuong_HCM_sparse_vector,
#   namespace="tu-tuong-HCM"
# )

Số trang trong PDF: 150
✅ Đã lưu 211 chunks vào: ./chunks/TT_HCM_Dense.json


In [ ]:
# Hàm dùng để lấy mẫu các text có kích thước khác nhau trong file PDF
# from collections import defaultdict

# def sample_unique_texts_by_size(pdf_path, total_pages, max_samples=5):
#     samples = defaultdict(list) 
#     seen = defaultdict(set) # Để theo dõi các dòng đã thấy cho mỗi kích thước
#     for page_layout in extract_pages(pdf_path, page_numbers=range(5, total_pages)):
#         for element in page_layout:
#             if isinstance(element, LTTextContainer):
#                 for text_line in element:
#                     if isinstance(text_line, LTTextLine):
#                         line_text = text_line.get_text().strip()
#                         for obj in text_line:
#                             if isinstance(obj, LTChar):
#                                 text_size = obj.size
#                                 # Chỉ lấy mẫu nếu chưa đủ số mẫu và chưa thấy dòng này với kích thước này
#                                 if (line_text not in seen[text_size] and 
#                                     len(samples[text_size]) < max_samples and 
#                                     line_text):
#                                     samples[text_size].append(line_text)
#                                     seen[text_size].add(line_text)
#     # In ra các mẫu đã lấy
#     for size in sorted(samples):
#         print(f"Size: {size}")
#         for example in samples[size]:
#             print(f"  {example}")
#         print("-" * 40)

# sample_unique_texts_by_size(pdf_path, total_pages)

Size: 1.0
  lOMoARcPSD|38213158
----------------------------------------
Size: 5.799999999999997
  1 Đảng Cộng sản Việt Nam: Văn kiện Đại hội đại biểu toàn quốc lần thứ XI, Nxb Chính trị quốc gia, Hà
  3 Đảng Cộng sản Việt Nam: Văn kiện Đảng Toàn tập, Nxb Chính trị quốc gia, Hà Nội, 2004, t.37, tr.474.
  4Đảng Cộng sản Việt Nam: Văn kiện Đại hội đại biểu toàn quốc lần thứ V, Nxb Sự thật, Hà Nội, t. 3, tr.61.
  4 Đảng Cộng sản Việt Nam: Văn kiện Đại hội đại biểu toàn quốc lần thứ IX, Nxb Chính trị quốc gia, Hà
  3  Xem  GS,TS  Mạch  Quang   Thắng,  PGS,TS  Bùi  Đình  Phong,  TS   Chu Đức  Tính  (Đồng  Chủ  biên):
----------------------------------------
Size: 5.800000000000011
  1 Đảng Cộng sản Việt Nam: Văn kiện Đảng Toàn tập, Nxb Chính trị quốc gia, Hà Nội, 2001, t.12, tr. 9.
  2 Đảng Cộng sản Việt Nam: Văn kiện Đảng Toàn tập, Nxb Chính trị quốc gia, Hà Nội, 2004, t.30, tr.275.
  1Đảng Cộng sản Việt Nam: Văn kiện Đảng Toàn tập, Nxb Chính trị quốc gia, Hà Nội, 2006,  t.47, tr.807.
  2 